In [8]:
from functools import reduce
from typing import List

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Add
from tensorflow.keras.models import Model

from interact.fields import FieldsManager, SparseField
from interact.layers import SparseLinear, AddBias
from interact.models import FM
from interact.utils import to_sequences

from utils import get_dataset, DataSet, cost, MercariTranformer

In [2]:
train = get_dataset(DataSet.Train)
valid = get_dataset(DataSet.Valid)

In [3]:
mercari_transformer = MercariTranformer(
    CountVectorizer(min_df=10),
    CountVectorizer(min_df=10),
)

In [4]:
%%time
mercari_transformer.fit(pd.concat([train, valid], axis=0, ignore_index=True))

CPU times: user 1min, sys: 1.26 s, total: 1min 1s
Wall time: 1min 1s


In [5]:
%%time
d_train = mercari_transformer.transform(train)

CPU times: user 2min 3s, sys: 560 ms, total: 2min 3s
Wall time: 2min 3s


In [6]:
%%time
d_valid = mercari_transformer.transform(valid)

CPU times: user 22.5 s, sys: 4.05 ms, total: 22.5 s
Wall time: 22.5 s


In [7]:
f_name = SparseField(
    name='name', 
    vocabulary_size=16543,
    m=10, 
    d=5,
)

f_description = SparseField(
    name='description', 
    vocabulary_size=30164,
    m=169, 
    d=5,
)

f_brand = SparseField(
    name='brand', 
    vocabulary_size=4658,
    m=1, 
    d=5,
)

f_condition = SparseField(
    name='item_condition_id', 
    vocabulary_size=5,
    m=1, 
    d=5,
)

f_shipping = SparseField(
    name='shipping', 
    vocabulary_size=2,
    m=1, 
    d=5,
)

f_category = SparseField(
    name='category', 
    vocabulary_size=946,
    m=5, 
    d=5,
)

In [10]:
fields = [
    f_name,
    f_description,
    f_brand,
    f_condition,
    f_shipping,
    f_category
]

In [11]:
fm = FM(fields, l2_penalty=0.001)

In [12]:
fm.compile(optimizer='sgd', loss='mse')

In [13]:
train_inputs = []
train_inputs.append(to_sequences(d_train["name"], 10))
train_inputs.append(to_sequences(d_train["item_description"], 169))
train_inputs.append(to_sequences(d_train["brand_name"], 1))
train_inputs.append((d_train["item_condition_id"].argmax(axis=1) + 1).reshape((-1, 1)))
train_inputs.append((d_train["shipping"] + 1).values.reshape((-1, 1)))
train_inputs.append(d_train["category_name"])

In [14]:
valid_inputs = []
valid_inputs.append(to_sequences(d_valid["name"], 10))
valid_inputs.append(to_sequences(d_valid["item_description"], 169))
valid_inputs.append(to_sequences(d_valid["brand_name"], 1))
valid_inputs.append((d_valid["item_condition_id"].argmax(axis=1) + 1).reshape((-1, 1)))
valid_inputs.append((d_valid["shipping"] + 1).values.reshape((-1, 1)))
valid_inputs.append(d_valid["category_name"])

In [15]:
fm.fit(
    train_inputs, 
    np.log1p(train['price']),
    epochs=10,
    batch_size=32,
    shuffle=True,
    validation_data=(
        valid_inputs,
        np.log1p(valid['price'])
    ), 
    callbacks=[EarlyStopping()]
)

Train on 1082535 samples, validate on 200000 samples
Epoch 1/10


/home/sigmo/deep/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1082535/1082535 [==============================] - 133s 123us/sample - loss: 0.4143 - val_loss: 0.3245
Epoch 2/10
1082535/1082535 [==============================] - 134s 124us/sample - loss: 0.2923 - val_loss: 0.2768
Epoch 3/10
1082535/1082535 [==============================] - 125s 115us/sample - loss: 0.2649 - val_loss: 0.2705
Epoch 4/10
1082535/1082535 [==============================] - 124s 115us/sample - loss: 0.2576 - val_loss: 0.2600
Epoch 5/10
1082535/1082535 [==============================] - 113s 104us/sample - loss: 0.2556 - val_loss: 0.2599
Epoch 6/10
1082535/1082535 [==============================] - 107s 99us/sample - loss: 0.2548 - val_loss: 0.2577
Epoch 7/10
1082535/1082535 [==============================] - 107s 99us/sample - loss: 0.2544 - val_loss: 0.2581


In [17]:
y_pred = np.expm1(fm.predict(valid_inputs)).flatten()

In [18]:
cost(true=valid['price'], pred=y_pred)

0.4802458715492957